In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-04-28 11:20:52--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.08s   

2025-04-28 11:20:52 (315 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
raw = df_ratings.copy()
users = raw['user'].value_counts()[lambda v: v>=200].index
books = raw['isbn'].value_counts()[lambda v: v>=100].index
filtered = raw[raw['user'].isin(users) & raw['isbn'].isin(books)]

# pivot to user×book matrix of raw ratings
user_book = filtered.pivot(index='user', columns='isbn', values='rating').fillna(0)

In [5]:
mat = csr_matrix(user_book.values)
knn = NearestNeighbors(n_neighbors=6, metric='cosine').fit(mat.T)

In [6]:
def get_recommends(book=""):
    if book not in df_books['title'].values:
        return "Book not found."
    isbn = df_books.loc[df_books['title']==book, 'isbn'].iloc[0]
    if isbn not in user_book.columns:
        return "Book not rated."
    col = user_book.columns.get_loc(isbn)
    dists, inds = knn.kneighbors(user_book.iloc[:,col].values.reshape(1,-1))
    pairs = sorted(zip(inds[0][1:], dists[0][1:]), key=lambda x: x[1], reverse=True)[:5]
    recommended_books = [
        [df_books.loc[df_books['isbn']==user_book.columns[i],'title'].iloc[0],
         float(round(dist,2))]
        for i,dist in pairs
    ]
    return [book, recommended_books]

In [7]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.800000011920929], ['The Weight of Water', 0.7699999809265137], ['The Surgeon', 0.7699999809265137], ['I Know This Much Is True', 0.7699999809265137], ['The Lovely Bones: A Novel', 0.7200000286102295]]]
You passed the challenge! 🎉🎉🎉🎉🎉
